In [1]:
import numpy as np
import pandas as pd
import re
from konlpy.tag import Mecab
import pymongo

In [2]:
host = '{SERVER IP}'
port = 27010

In [3]:
conn = pymongo.MongoClient(host=host, port=port)
db = conn.get_database('news')  ## db name
collection = db.get_collection('marine')

In [4]:
news_marine = list(
    collection.find({'service_type': 'NEWS', '$text': {'$search': '해운 항만 조선 해양 해운업 조선업 해양수산 -주식 -호텔'}}, {'_id': 0, 'content': 1}))
news_content = [news['content'] for news in news_marine]
news_data = pd.DataFrame(news_content)

print('MongoDB Connected.')

MongoDB Connected.


In [5]:
conn.close()
print('MongoDB Closed.')

MongoDB Closed.


In [6]:
news_data.head(10)
len(news_data)

164371

In [7]:
news_train = news_data.rename(columns={0: 'news'})
news_train['news'][:30]

0     증권\n기사\n입력 : 2020-11-23 17:51:07\n▲ 23일 조선 테마가...
1     [서울=뉴시스]웨스틴 조선 '조선 델리'의 '로맨틱 럭셔리'\n3월14일은 '화이트...
2     LNG운반선.©대우조선해양\n국내 조선 빅3인 대우조선해양, 현대중공업, 삼성중공업...
3     LNG운반선.©대우조선해양\n국내 조선 빅3인 대우조선해양, 현대중공업, 삼성중공업...
4     (서울=연합뉴스) 김보경 기자 = 해양수산부와 국립해양박물관은 16일부터 내년 2월...
5     해군사관학교 해양연구소 송기중 연구위원.\n[내외일보=경남] 김성삼 기자 = 해군사...
6     [헤럴드DB]\n[헤럴드경제=김대우 기자]조선 업황 회복 분위기 속에 국내 주요 조...
7     [헤럴드DB]\n[헤럴드경제=김대우 기자]조선 업황 회복 분위기 속에 국내 주요 조...
8     조선호텔, 내달 25일 해운대에 '그랜드 조선 부산' 개관\n(서울=연합뉴스) 박상...
9     '세계 1위' 조선의 역사 엿보다…해수부, 조선시대 선박 전시회\n국립해양박물관서 ...
10    송고시간 | 2019-09-23 10:00\n조선업[연합뉴스TV 제공]\n(세종=연...
11    한국기업평판연구소는 조선 상장기업 브랜드평판 조사결과, 한국조선해양이 1위에 올랐다...
12    해수부·해양박물관, '조선(造船)의 나라, 조선(朝鮮)' 전시회 개최\n등록 202...
13    ▲김성갑 경상남도의회 경제환경위원회 위원장\n중국에서 시작하여 아시아 지역은 물론 ...
14    ◀ＡＮＣ▶\n조선 경기가 회복세를 보이고 있지만\n수년간 이어진 인력 감축의 여파로...
15    9일 ‘조선해양의 날’ 맞아 경남 거제서 K-조선 비전 선언\n문재인 대통령이 9일...
16    문 대통령, 세계 1등 'K-조선' 현장 방문…조선강국 재도약 전략 논의\n입력: ...
17    [여성소비자신문 이지은 기자]조선 상장기업 브랜드평판 2021년 11월 빅데

In [8]:
dup = news_train.duplicated(['news']) # 여기서 dup의 데이터타입은 Series
dup


0         False
1         False
2         False
3          True
4         False
          ...  
164366     True
164367     True
164368    False
164369    False
164370    False
Length: 164371, dtype: bool

In [9]:
news_train.news = news_train.news.astype(str)

In [10]:
news_train = news_train.drop_duplicates(['news'])

In [11]:
news_train

,news
0,증권\n기사\n입력 : 2020-11-23 17:51:07\n▲ 23일 조선 테마가...
1,[서울=뉴시스]웨스틴 조선 '조선 델리'의 '로맨틱 럭셔리'\n3월14일은 '화이트...
2,"LNG운반선.©대우조선해양\n국내 조선 빅3인 대우조선해양, 현대중공업, 삼성중공업..."
4,(서울=연합뉴스) 김보경 기자 = 해양수산부와 국립해양박물관은 16일부터 내년 2월...
5,해군사관학교 해양연구소 송기중 연구위원.\n[내외일보=경남] 김성삼 기자 = 해군사...
...,...
164364,국제 정세 속 한국은? 마입성 쌍로지상(馬入城 雙路之像)!…亥는 쌍해종환(雙亥終還)...
164365,1975년 미국 코닥사의 엔지니어 스티븐 세슨은 세계 최초로 디지털카메라를 발명했습...
164368,1975년 미국 코닥사의 엔지니어 스티븐 세슨은 세계 최초로 디지털카메라를 발명했습...
164369,지금부터 제51차 비상경제 중대본회의 정례브리핑을 시작하겠습니다.\n오늘 회의는 2...


In [12]:
news_train['news'] = news_train['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣|a-zA-Z]","")

/var/folders/9c/dyfvcxrx45d7q0h1vnt_h6p40000gn/T/ipykernel_1570/4061227587.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  news_train['news'] = news_train['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣|a-zA-Z]","")


In [13]:
data_list = news_train.news.values.tolist()

In [14]:
tokenizer = Mecab(dicpath='/Users/yhkoo/mecab-ko-dic-2.0.3-20170922')

In [15]:
print(data_list[0])

증권기사입력일조선테마가급등세로마감했다이날조선테마는전장거래일대비급등했다조선테마구성종목에는삼성중공업현대미포조선한국조선해양대우조선해양한전중공업이있다사진연합뉴스베타뉴스이춘희기자일조선테마가급등세로마감했다이날조선테마는전장거래일대비급등했다조선테마구성종목에는삼성중공업현대미포조선한국조선해양대우조선해양한전중공업이있다이가운데삼성중공업은를기록하며조선테마상승을견인했다이어서현대미포조선한국조선해양대우조선해양한진중공업를기록했다베타뉴스이춘희기자pressbetanewsnetCopyrightsBetaNewsnet목록위로


In [16]:
data_word = []
for i in range(len(data_list)):
    tagged = tokenizer.pos(data_list[i])
    nouns = [s for s, t in tagged if t in ['SL','NNG','NNP'] and len(s) > 1]
    data_word.append(nouns)

In [17]:
print(data_word[251])

['에너지', '가격', '상승', '수주', '낭보', '조선', '업종', '반등', '내년도', '라운드', '기대', '피크', '아웃', '고점', '통과', '해운', '업종', '조선', '업종', '투자', '매력', '전망', '조선', '업종', '동반', '강세', '날삼', '중공업', '현대', '중공업', '대우', '조선', '해양', '현대', '미포', '조선', '조선', '업종', '전반', '상승', '마감', '조선', '업종', '상승세', '날삼', '중공업', '버뮤다', '지역', '선사', '규모', '액화천연가스', 'LNG', '운반', '선척', '수주', '공시', '회사', '셔틀', '탱커', '수주', '주일', '새조', '수주', '실적', '이날', '한국', '조선', '해양', '자회사', '현대', '중공업', '중동', '지역', '선주', '석유', '화학', '운반', 'PC', '선척', '수주', '발표', '대우', '조선', '해양', '지역', '선주', 'LNG', '운반', '선척', '수주', '공시', '신한', '금융', '투자', 'LNG', '운반선', '탱커', '원유', '석유', '화학제품', '운반', '내년', '인상', '견인', '전망', 'LNG', '가격', '겨울', '에너지', '수요', '친환경', '에너지', '전환', '화석', '연료', '탄소', '배출', '가스', '소비량', '급증', '수요', '공급', '제한', 'LNG', '가격', '고공', '행진', '분기', '평균', 'LNG', '가격', '전년', '동기', '대비', '상승', 'LNG', '운반선', '발주', '환경', 'LNG', '운반선', '발주', '예상', '프로젝트', '카타르', '노스', '필드', '프로젝트', '내년', '모잠비크', 'LNG', '프로젝트', '내년', '상반기', '러시아', '발틱', 'LNG', '프로젝트', '조선',

In [18]:
#LDA학습을 위한 객체생성
from gensim import corpora

id2word = corpora.Dictionary(data_word)
id2word.filter_extremes(no_below=10)
texts = data_word
corpus = [id2word.doc2bow(text) for text in texts]

In [19]:
import gensim
NUM_TOPICS = 100
printset = []

ldamodel = gensim.models.LdaModel(corpus=corpus,num_topics=NUM_TOPICS, id2word=id2word,passes=15, random_state=777)
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    printset.append(topic)

In [25]:
print(topics)

[(79, '0.305*"비주얼" + 0.147*"플러스" + 0.121*"단장" + 0.077*"동아" + 0.049*"애플" + 0.042*"사업단" + 0.038*"합성" + 0.022*"지사장" + 0.017*"그래프" + 0.016*"학장"'), (34, '0.030*"한진" + 0.028*"인수" + 0.021*"자금" + 0.017*"기업" + 0.016*"매각" + 0.015*"구조" + 0.012*"회사" + 0.012*"경영" + 0.012*"산업은행" + 0.011*"자산"'), (27, '0.047*"SK" + 0.045*"바이오" + 0.037*"삼성전자" + 0.034*"LG" + 0.021*"반도체" + 0.015*"CJ" + 0.014*"삼성" + 0.014*"카카오" + 0.014*"화학" + 0.014*"전자"'), (40, '0.053*"분기" + 0.045*"영업" + 0.038*"이익" + 0.034*"실적" + 0.032*"증가" + 0.030*"대비" + 0.029*"올해" + 0.028*"매출" + 0.028*"지난해" + 0.024*"전년"'), (81, '0.049*"행위" + 0.041*"수사" + 0.035*"공정위" + 0.034*"공정" + 0.029*"불법" + 0.028*"사건" + 0.028*"조사" + 0.026*"제재" + 0.019*"위반" + 0.018*"해운법"'), (49, '0.615*"뉴스" + 0.076*"수정" + 0.055*"보도" + 0.047*"전체" + 0.032*"입력" + 0.028*"앵커" + 0.028*"영상" + 0.020*"시간" + 0.020*"오늘" + 0.010*"리포트"'), (87, '0.278*"연안" + 0.237*"세월" + 0.138*"여객선" + 0.084*"승객" + 0.077*"화물선" + 0.065*"여객" + 0.025*"부기" + 0.016*"초대형" + 0.012*"서민" + 0.009*"선척"'), (93, '0.035*"발레" + 0.

In [23]:
df = pd.DataFrame(printset)

In [24]:
df

,0,1
0,79,"0.305*""비주얼"" + 0.147*""플러스"" + 0.121*""단장"" + 0.077..."
1,34,"0.030*""한진"" + 0.028*""인수"" + 0.021*""자금"" + 0.017*""..."
2,27,"0.047*""SK"" + 0.045*""바이오"" + 0.037*""삼성전자"" + 0.03..."
3,40,"0.053*""분기"" + 0.045*""영업"" + 0.038*""이익"" + 0.034*""..."
4,81,"0.049*""행위"" + 0.041*""수사"" + 0.035*""공정위"" + 0.034*..."
5,49,"0.615*""뉴스"" + 0.076*""수정"" + 0.055*""보도"" + 0.047*""..."
6,87,"0.278*""연안"" + 0.237*""세월"" + 0.138*""여객선"" + 0.084*..."
7,93,"0.035*""발레"" + 0.031*""모바일"" + 0.030*""콘텐츠"" + 0.029..."
8,10,"0.000*""도초도"" + 0.000*""농대"" + 0.000*""종재"" + 0.000*..."
9,30,"0.151*""부경"" + 0.139*""실습"" + 0.103*""스틸"" + 0.098*""..."


In [26]:
lpt = ldamodel.print_topics(num_topics=100, num_words=10)

In [30]:
lpt_df = pd.DataFrame(lpt)
ll = lpt_df.drop(columns=0)

In [33]:
ll_df = ll.rename(columns={1: '연관키워드'})
ll_df

,연관키워드
0,"0.022*""지역"" + 0.021*""추진"" + 0.020*""산업"" + 0.019*""..."
1,"0.237*""항공"" + 0.098*""공항"" + 0.058*""항공사"" + 0.058*..."
2,"0.323*""저널"" + 0.173*""작물"" + 0.144*""통장"" + 0.131*""..."
3,"0.019*""후보"" + 0.014*""정치"" + 0.014*""민주당"" + 0.012*..."
4,"0.565*""유엔"" + 0.102*""진원"" + 0.089*""밸리"" + 0.039*""..."
...,...
95,"0.453*""주일"" + 0.213*""식수"" + 0.181*""원상"" + 0.000*""..."
96,"0.484*""자원"" + 0.132*""활용"" + 0.122*""순환"" + 0.102*""..."
97,"0.029*""판매"" + 0.023*""상품"" + 0.021*""제품"" + 0.021*""..."
98,"0.158*""금융"" + 0.061*""은행"" + 0.058*""투자"" + 0.049*""..."


In [35]:
ll_df.to_csv("./lda_marine(19-21).csv", mode='w')

In [56]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.27748343), (17, 0.6659657), (19, 0.045959007)]
1 번째 문서의 topic 비율은 [(1, 0.05430766), (4, 0.037825696), (5, 0.017865071), (17, 0.11871424), (18, 0.108769596), (21, 0.03019154), (25, 0.47794902), (26, 0.031940255), (27, 0.048836093), (29, 0.0380671)]
2 번째 문서의 topic 비율은 [(0, 0.13870952), (17, 0.73702216), (24, 0.08691222), (28, 0.030957613)]
3 번째 문서의 topic 비율은 [(1, 0.026097644), (11, 0.10751713), (17, 0.18525884), (18, 0.18471228), (24, 0.08447206), (25, 0.29464912), (26, 0.10304267)]
4 번째 문서의 topic 비율은 [(2, 0.02729411), (11, 0.1882472), (17, 0.028834265), (18, 0.32785425), (20, 0.028845327), (21, 0.021494621), (25, 0.20394881), (26, 0.14563121), (28, 0.010127118)]


In [57]:
ldamodel.s .show_topic(5, topn=20)

[('제품', 0.042817518),
 ('사용', 0.026955176),
 ('식품', 0.026630742),
 ('플라스틱', 0.026243828),
 ('소재', 0.01776406),
 ('생산', 0.015270778),
 ('용기', 0.012305038),
 ('판매', 0.01185771),
 ('브랜드', 0.011285035),
 ('택배', 0.01063551)]

In [58]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, row in enumerate(ldamodel[corpus]):
        # doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(row, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%),
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                wp = ldamodel.show_topic(topic_num,topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [59]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', ' 연관어']
topictable[:10]

/var/folders/9c/dyfvcxrx45d7q0h1vnt_h6p40000gn/T/ipykernel_930/3622223429.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
/var/folders/9c/dyfvcxrx45d7q0h1vnt_h6p40000gn/T/ipykernel_930/3622223429.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,연관어
0,0,17,0.6659,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"
1,1,25,0.4779,"영상, 바다, 시간, 뉴스, 기사, 사람, 문화, 제주, 월일, 사진"
2,2,17,0.7370,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"
3,3,25,0.2947,"영상, 바다, 시간, 뉴스, 기사, 사람, 문화, 제주, 월일, 사진"
4,4,18,0.3279,"산업, 지원, 분야, 기술, 사업, 기업, 개발, 혁신, 연구, 협력"
5,5,17,0.2893,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"
6,6,17,0.3358,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"
7,7,25,0.2920,"영상, 바다, 시간, 뉴스, 기사, 사람, 문화, 제주, 월일, 사진"
8,8,17,0.2993,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"
9,9,17,0.4301,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"


In [60]:
topictable[:100]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,연관어
0,0,17,0.6659,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"
1,1,25,0.4779,"영상, 바다, 시간, 뉴스, 기사, 사람, 문화, 제주, 월일, 사진"
2,2,17,0.7370,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"
3,3,25,0.2947,"영상, 바다, 시간, 뉴스, 기사, 사람, 문화, 제주, 월일, 사진"
4,4,18,0.3279,"산업, 지원, 분야, 기술, 사업, 기업, 개발, 혁신, 연구, 협력"
...,...,...,...,...
95,95,19,0.3200,"기업, 현대, 그룹, 투자, 금융, 시장, 회장, 사업, 인수, 글로벌"
96,96,19,0.2989,"기업, 현대, 그룹, 투자, 금융, 시장, 회장, 사업, 인수, 글로벌"
97,97,18,0.5553,"산업, 지원, 분야, 기술, 사업, 기업, 개발, 혁신, 연구, 협력"
98,98,14,0.4852,"기술, 시스템, 데이터, 안전, 개발, 정보, 서비스, 디지털, 활용, 선박"


import warnings
warnings..filterwarnings(action = 'ignore')
warning 무시하기

In [71]:
dfdf = topictable.drop_duplicates(['가장 비중이 높은 토픽','가장 높은 토픽의 비중',' 연관어'], keep='first')
dfdf

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,연관어
0,0,17,0.6659,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"
1,1,25,0.4779,"영상, 바다, 시간, 뉴스, 기사, 사람, 문화, 제주, 월일, 사진"
2,2,17,0.7370,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"
3,3,25,0.2947,"영상, 바다, 시간, 뉴스, 기사, 사람, 문화, 제주, 월일, 사진"
4,4,18,0.3279,"산업, 지원, 분야, 기술, 사업, 기업, 개발, 혁신, 연구, 협력"
...,...,...,...,...
144571,144571,18,0.2344,"산업, 지원, 분야, 기술, 사업, 기업, 개발, 혁신, 연구, 협력"
144583,144583,19,0.5237,"기업, 현대, 그룹, 투자, 금융, 시장, 회장, 사업, 인수, 글로벌"
144584,144584,19,0.5232,"기업, 현대, 그룹, 투자, 금융, 시장, 회장, 사업, 인수, 글로벌"
144587,144587,19,0.2912,"기업, 현대, 그룹, 투자, 금융, 시장, 회장, 사업, 인수, 글로벌"


In [72]:
df1 = dfdf.drop('문서 번호', axis=1)
df2 = df1.reset_index(drop=True)

In [73]:
df2

,가장 비중이 높은 토픽,가장 높은 토픽의 비중,연관어
0,17,0.6659,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"
1,25,0.4779,"영상, 바다, 시간, 뉴스, 기사, 사람, 문화, 제주, 월일, 사진"
2,17,0.7370,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"
3,25,0.2947,"영상, 바다, 시간, 뉴스, 기사, 사람, 문화, 제주, 월일, 사진"
4,18,0.3279,"산업, 지원, 분야, 기술, 사업, 기업, 개발, 혁신, 연구, 협력"
...,...,...,...
65468,18,0.2344,"산업, 지원, 분야, 기술, 사업, 기업, 개발, 혁신, 연구, 협력"
65469,19,0.5237,"기업, 현대, 그룹, 투자, 금융, 시장, 회장, 사업, 인수, 글로벌"
65470,19,0.5232,"기업, 현대, 그룹, 투자, 금융, 시장, 회장, 사업, 인수, 글로벌"
65471,19,0.2912,"기업, 현대, 그룹, 투자, 금융, 시장, 회장, 사업, 인수, 글로벌"


In [75]:
df2.to_csv("./ldaresult.csv", mode='w')

In [69]:
dld= df1.drop_duplicates(['가장 비중이 높은 토픽',' 연관어'], keep='first')
dld

,가장 비중이 높은 토픽,가장 높은 토픽의 비중,연관어
0,17,0.6659,"상승, 전망, 세계, 가격, 예상, 수준, 조선, 기록, 최근, 운임"
1,25,0.4779,"영상, 바다, 시간, 뉴스, 기사, 사람, 문화, 제주, 월일, 사진"
4,18,0.3279,"산업, 지원, 분야, 기술, 사업, 기업, 개발, 혁신, 연구, 협력"
19,19,0.3294,"기업, 현대, 그룹, 투자, 금융, 시장, 회장, 사업, 인수, 글로벌"
27,22,0.5631,"노조, 관련, 한국, 행위, 임금, 정부, 결정, 한진, 수사, 사건"
84,14,0.4641,"기술, 시스템, 데이터, 안전, 개발, 정보, 서비스, 디지털, 활용, 선박"
88,24,0.2956,"선박, 오염, LNG, 규제, 연료, 원전, 환경, 처리, 설비, 정부"
90,10,0.3400,"분기, 영업, 이익, 증가, 실적, 대비, 지난해, 매출, 올해, 전년"
112,26,0.2552,"중국, 미국, 일본, 한국, 국가, 세계, 나라, 홍콩, 유럽, 영국"
144,5,0.3896,"제품, 사용, 식품, 플라스틱, 소재, 생산, 용기, 판매, 브랜드, 택배"
